In [2]:
import pandas as pd

df = pd.read_csv('owid-covid-data.csv')

In [3]:
top10_vaccinated_iso_codes = df.loc[df.population > 1000000, :] \
  .loc[df.iso_code != 'OWID_NAM', :] \
  .dropna(subset=['people_vaccinated_per_hundred']) \
  .groupby('iso_code') \
  .tail(1)[["date","iso_code","location", "people_vaccinated_per_hundred"]] \
  .sort_values('people_vaccinated_per_hundred') \
  .tail(10)['iso_code'].tolist()
top10_vaccinated_iso_codes

['CAN', 'SRB', 'URY', 'HUN', 'BHR', 'USA', 'CHL', 'GBR', 'ARE', 'ISR']

In [4]:
import altair as alt

target = df.loc[df.iso_code.isin(top10_vaccinated_iso_codes) , :].dropna(subset=['people_vaccinated_per_hundred'])

dropdown = alt.binding_select(options=target['location'].unique(), name='Select a location:')

dropdown_selection = alt.selection(type='single', fields=['location'], bind=dropdown)

selection = alt.selection(type='multi', fields=['location'])

hosp = alt.Chart(target).mark_line().encode(
    alt.X('yearmonthdate(date)', title='date'),
    y = 'hosp_patients_per_million',
    color='location',
    opacity=alt.condition(selection | dropdown_selection, alt.value(1), alt.value(.2))
).add_selection(
    selection, dropdown_selection
)

vaccine = alt.Chart(target).mark_line(strokeDash=[1,1]).encode(
    alt.X('yearmonthdate(date)', title='date'),
    alt.Y('y:Q', axis=alt.Axis(format='%'), title='percent of vaccinated'),
    color='location',
    opacity=alt.condition(selection | dropdown_selection, alt.value(1), alt.value(.2))
).transform_calculate(
    y = 'datum.people_vaccinated_per_hundred * 0.01'
).add_selection(
    selection, dropdown_selection
)

vaccine | hosp


alt.HConcatChart(...)

In [5]:
from vega_datasets import data
from iso3166 import countries

df = pd.read_csv('owid-covid-data.csv')

def map_country_code(x):
    try:
        code = countries.get(x).numeric
        return code
    except:
        return None

df['id'] = df['iso_code'].map(map_country_code)

source = alt.topo_feature(data.world_110m.url, 'countries')

# target = df.dropna(subset=['id','hosp_patients_per_million']).drop_duplicates(subset=['iso_code'], keep='last')



df['timestamp'] = [pd.to_datetime(d).timestamp() for d in df['date']]

# slider = alt.binding_range(
#     min=min(df['timestamp']),
#     max=max(df['timestamp'])
# )

# select_date = alt.selection_single(
#     name='slider',
#     fields=['timestamp'],
#     bind=slider,
#     init={"timestamp": pd.to_datetime('2020-04-20').timestamp()}
# )

# partial_df = df.loc[df['date'] == '2020-04-20', :]

partial_df = df.drop_duplicates(subset=['iso_code'], keep='last')

print(partial_df[['date', 'iso_code', 'id', 'hosp_patients_per_million']])

alt.Chart(partial_df).mark_geoshape().encode(
    color='hosp_patients_per_million:Q'
# ).add_selection(
#     select_date
# ).transform_filter(
#     select_date
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(source, key='id', fields=["type", "properties", "geometry"])
).properties(
    width=500,
    height=300
).project(
    type='equirectangular'
)


             date  iso_code    id  hosp_patients_per_million
421    2021-04-20       AFG   004                        NaN
854    2021-04-20  OWID_AFR  None                        NaN
1275   2021-04-20       ALB   008                        NaN
1696   2021-04-20       DZA   012                        NaN
2111   2021-04-20       AND   020                        NaN
...           ...       ...   ...                        ...
81361  2021-04-20       VNM   704                        NaN
81816  2021-04-20  OWID_WRL  None                        NaN
82192  2021-04-20       YEM   887                        NaN
82591  2021-04-20       ZMB   894                        NaN
82988  2021-04-20       ZWE   716                        NaN

[219 rows x 4 columns]


alt.Chart(...)